In [110]:
import pandas as pd
df = pd.read_csv("/Users/janbol/Documents/GitHub/RAG/data/test.csv")
df= df[df["Result"].notna()]
data = df.to_dict("records")

In [112]:
len(data)

653

In [119]:
data

[{'Summary': 'on august 27 2013 employees #1 and #2 of templar inc. a construction company specializing in fiber optic installation and services were working along a highway. the highway speed limit was posted at 55 miles per hour. employee #1 was marking the location of an underground line that ran below the turn lane. employee #2 was next to employee #1 and performing the duties of a flagger. a privately owned vehicle was travelling in the travel/through lane. the vehicle veered to the right entered the turn lane and struck both workers. emergency medical services were called. employee #1 was declared dead at the scene. employee #2 refused emergency medical treatment for the bruises he received when struck.',
  'Result': 1},
 {'Summary': 'at approximately 6:30 a.m. on may 13 2013 employee #1 a foreman regularly employed by integrity rebar placers was operating a rough terrain forklift at a contracted job site in murrieta ca. he was using the forklift to move bundles of steel. employe

In [113]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [114]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [115]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [117]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="incidents",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/var/folders/c5/p6q0gjzj2ns5lv27n9yp07ph0000gn/T/ipykernel_39385/2185663414.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [122]:
# vectorize!
qdrant.upload_points(
    collection_name="incidents",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Summary"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [134]:
user_prompt = "incidents involving water and rain"

In [ ]:
import json
hits = qdrant.search(
    collection_name="incidents",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
) 
for hit in hits:
  print(json.dumps(hit.payload, indent=4), "score:", hit.score)

{
    "Summary": "on may 9 2000 employee #1 was performing abrasive blasting operations on the roof of a water tower. employee #1 had not donned his fall protection equipment and was subsequently not tied off. when he got to the edge of the tower he slipped or stumbled and fell 150 ft to the ground. employee #1 was killed.",
    "Result": 1
} score: 0.4744931897244873
{
    "Summary": "on october 4 2005 employee #1 was working as a carpenter specializing in building docks. he was installing fender panels on a bridge pier. as he pulled a 1 inch diameter air hose into position to power some pneumatic tools he fell from the pier into water approximately fifteen feet deep and drowned. possibly he struck his head on a protruding portion of the fender system prior to striking the water.",
    "Result": 1
} score: 0.4404670333272622
{
    "Summary": "on march 16 2001 employee #1 and a coworker were doing abrasive blasting from separate man lifts inside a hydro-pillar water tank. suddenly the 

In [136]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [148]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are a bot, an construction safety engineer. Your top priority is to help guide fellow engineers to follow safety protocols to reduce incidents at workplace"},
        {"role": "user", "content": "how to work safely in slippery and wet surfaces"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
output = completion.choices[0].message.to_dict()
print(output["content"])


As a construction safety engineer, I'd like to emphasize the importance of working safely on slippery and wet surfaces. Here are some guidelines to help prevent accidents:

**Pre-Job Precautions**

1. **Conduct a hazard assessment**: Identify potential slippery and wet areas on the job site, such as floors, stairs, and walkways.
2. **Clean up spills immediately**: Ensure that any spills or leaks are cleaned up promptly to prevent slipping hazards.
3. **Use warning signs**: Post warning signs in areas with slippery and wet surfaces to alert workers of the potential hazard.

**Personal Protective Equipment (PPE)**

1. **Wear non-slip shoes**: Ensure that workers wear non-slip shoes or boots with good traction to prevent slipping on wet surfaces.
2. **Use slip-resistant gloves**: Provide workers with slip-resistant gloves to prevent slipping on wet surfaces.
3. **Use safety harnesses**: Ensure that workers are properly tied off to a safety line when working on wet surfaces, such as roofs 